# Telangana RERA Scraper

1. Open the website ™

In [9]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# noinspection PyRedeclaration
browser = webdriver.Chrome(executable_path="/usr/local/bin/chromedriver") # TODO
browser.get("http://rerait.telangana.gov.in/SearchList/Search")
page_size = browser.find_element_by_id('PageSize')
page_size.send_keys("0")